# Semi-Interactive DFA Learning

*April 2018 PMC@ISCAS*

---

![logo](http://iscasmc.ios.ac.cn/roll/lib/exe/fetch.php?media=wiki:logo.png)

This is a tutorial and demonstration of our Java Library [```Regular Omega Language Learning (ROLL)```](http://iscasmc.ios.ac.cn/roll/doku.php) in a Groovy kernel Jupyter Notebook.You do not need to worry if you use Java and not familiar with Groovy. Because the syntax of these two languages are similar.

In this notebook you will see 
- how to write an function that answers membership query
- how to configure a Semi-Interactive learning process
- how to print the intermediate hypothesis and learner data structure in this hyper text environment
- how to create counter examples and use them to refine current hypothesis
- how to interactive with learning process 

You can run this tutorial step by step, or you can run your code and see what will happend.

**Tips** : If something goes strange, just use the menu bar above ```Kernel -> Restart & Clear Output``` to rebot this notebook.

---

**First we load our compiled single fat jar.**

In [1]:
%classpath add jar ROLL.jar

Added jar: [ROLL.jar]


**Chose an regular language in your mind**

For example, here we choose $\{ a^n \cdot b^m \mid n \ge 0 \text{ and } m \ge 1\}$.

**Then we creat the alphabet object $\{a, b\}$ of your target language**

In [2]:
import roll.words.Alphabet

alphabet = new Alphabet();
alphabet.addLetter((char)'a');
alphabet.addLetter((char)'b');

alphabet

roll.words.Alphabet@7b9f603d

**And write a piece of code to answer the membership querry of your target language**


In [3]:
memberAnswer_astartbplus = { w -> 
    boolean visb = false;
    for (int i = 0; i < w.length(); i++) {
        if (w.charAt(i) == 'a' && visb) {
            return false;
        } else if (w.charAt(i) == 'b') {
                visb = true;
        }
    }
    return visb;
};

script1524229644979499446995$_run_closure1@4cc649f2

You should guarantee that your implemention is consistent by your self. 

We can test it with several example:

In [4]:
System.out.println(memberAnswer_astartbplus(""))
System.out.println(memberAnswer_astartbplus("a"))
System.out.println(memberAnswer_astartbplus("ba"))
System.out.println(memberAnswer_astartbplus("aba"))

System.out.println(memberAnswer_astartbplus("b"))
System.out.println(memberAnswer_astartbplus("abbbbb"))

false
false
false
false
true
true


null

**Second we create an ```Options``` object to configure the learning process** 

For DFA we have 3 kinds of algorithm and 2 kinds data structure. But there are only 4 valid composition.

|                              | Options.Structure.TREE | Options.Structure.TABLE |
|:----------------------------:|:----------------------:|:-----------------------:|
| Options.Algorithm.DFA_LSTAR  | ✖️                     | ✔️                      |
| Options.Algorithm.DFA_KV     | ✔️                     | ✖️                      |
| Options.Algorithm.DFA_COLUMN | ✔️                     | ✔️                      |

Here we use a table based column algorithm as a demonstration.

In [5]:
import roll.main.*

options = new Options();
options.algorithm = Options.Algorithm.DFA_COLUMN;
options.structure = Options.Structure.TABLE;
options

LEARNING,TABLE,DFA_COLUMN,NONE,NBA,UNDER,verbose=false,bs=false,dot=false,inputfile=null,outputfile=null,outputA=null,outputB=null


**Then we create a SemiLearning process**


```SemiLearning(alphabet,options,functor)``` takes three arguments. ```alphabet``` is the alphabet of your target language. ```options``` configures wichi type of learning algorithm will be used. ```functor``` is a ```Function<String,Boolean>``` instance as membership oracle.

In [6]:
import roll.main.*
import java.util.function.Function;

Function<String,Boolean> functor = memberAnswer_astartbplus
l0 = new SemiLearning(alphabet,options,functor)

roll.main.SemiLearning@7f49dc50

In [7]:
l0.getHypothesis()

<?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 0 <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 1 <!-- 1->0 --> 1->0 0 <!-- 1->1 --> 1->1 1 <!-- 2 --> 2 <!-- 2->0 --> 2->0

We observed that $bab$ is in current hypothesis while it is not in our target language.

So we create an counter example object and use it to refine current hypothesis.

In [8]:
import roll.query.CE
counter_example = CE.withAlphabet(alphabet).finite("bab")
l0.refineHypothesis(counter_example)

Previous Hypothesis 
 Current Hypothesis 
 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 0 <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 1 <!-- 1->0 --> 1->0 0 <!-- 1->1 --> 1->1 1 <!-- 2 --> 2 <!-- 2->0 --> 2->0 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 0 <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 1 <!-- 1->1 --> 1->1 1 <!-- 2 --> 2 2 <!-- 1->2 --> 1->2 0 <!-- 2->2 --> 2->2 0 <!-- 2->2 --> 2->2 1 <!-- 3 --> 3 <!-- 3->0 --> 3->0

<span style="color:red">**Tips:** ```refineHypothesis``` has side effect.</span>

If you run code in last cell for twice, you will get an ```Exception``` since $bab$ is no longer being an valid counter example.

In [9]:
import roll.query.CE
l0.refineHypothesis(CE.withAlphabet(alphabet).finite("bab"))

Counter Example Error : bab:ϵ is neither in target or hypothesis.


java.lang.Exception:  Counter Example Error

---

**Try another language on $\{ a, b \}$**

For example, here we choose $\{ w \mid \text{ the number of } a \text{ occurs in } w \text{ is odd and the number of } b \text{ occurs in } w \text{ is odd} \}$.


In [10]:
memberAnswer_aoddbodd = { w -> 
        long cntA = w.chars().filter({x -> x == 'a'}).count();
        long cntB = w.chars().filter({x -> x == 'b'}).count();
        return (cntA % 2 == 1) && (cntB % 2 == 1) && (cntA + cntB == w.length());
}

script1524229646211939980920$_run_closure1@31ed8888

In [11]:
System.out.println(memberAnswer_aoddbodd(""))
System.out.println(memberAnswer_aoddbodd("a"))
System.out.println(memberAnswer_aoddbodd("b"))
System.out.println(memberAnswer_aoddbodd("abab"))

System.out.println(memberAnswer_aoddbodd("ab"))
System.out.println(memberAnswer_aoddbodd("abbbbb"))

false
false
false
false
true
true


null

In [12]:
import roll.main.*
import java.util.function.Function;

Function<String,Boolean> functor = memberAnswer_aoddbodd;
l1 = new SemiLearning(alphabet,options,functor)

roll.main.SemiLearning@50207d0f

In [13]:
l1.getHypothesis()

<?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 0 <!-- 0->0 --> 0->0 1 <!-- 1 --> 1 <!-- 1->0 --> 1->0

In [14]:
import roll.query.CE
l1.refineHypothesis(CE.withAlphabet(alphabet).finite("ab"))

Previous Hypothesis 
 Current Hypothesis 
 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 0 <!-- 0->0 --> 0->0 1 <!-- 1 --> 1 <!-- 1->0 --> 1->0 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd"><!-- Generated by graphviz version 2.38.0 (20140413.2041) --><!-- Title: %3 Pages: 1 --> %3 <!-- 0 --> 0 0 <!-- 0->0 --> 0->0 1 <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 0 <!-- 1->0 --> 1->0 0 <!-- 2 --> 2 2 <!-- 1->2 --> 1->2 1 <!-- 2->0 --> 2->0 0 <!-- 2->1 --> 2->1 1 <!-- 3 --> 3 <!-- 3->0 --> 3->0

In [15]:
import roll.query.CE
l1.refineHypothesis(CE.withAlphabet(alphabet).finite("bab"))

Previous Hypothesis,Current Hypothesis
